image


In [1]:
import numpy as np
import cv2 as cv


def find_corner(y_p, x_p, frame):
    y_offset =9
    x_offset =12
    y_new = 0
    x_new = 0
    
    window = frame[y_p-y_offset:y_p+y_offset, x_p-x_offset:x_p+x_offset]

    gray = cv.cvtColor(window,cv.COLOR_BGR2GRAY)
    #gray = np.float32(gray)

    corners = cv.goodFeaturesToTrack(gray,1,0.0015,5)
    corners = np.intp(corners)
    for i in corners:
        x,y = i.ravel()
        #new_frame[y + y_p-y_offset - 6 :y + y_p-y_offset +6, x+ x_p-x_offset -6:x+ x_p-x_offset +6 ]=[0,0,255]
        #new_frame = cv.circle(new_frame,(x+ x_p-x_offset ,y + y_p-y_offset),3,255,-1)
        y_new = y + y_p-y_offset
        x_new = x+ x_p-x_offset
    
    """
    dst = cv.cornerHarris(gray,2,3,0.04)
    #result is dilated for marking the corners, not important
    dst = cv.dilate(dst,None)
    # Threshold for an optimal value, it may vary depending on the image.
    my = dst>0.1*dst.max()
    
    for y in range(gray.shape[0]):
        for x in range(gray.shape[1]):
            if my[y][x]:
                #frame = cv.circle(frame, (x_p + x-x_offset,y_p+y-y_offset), 100, (0, 0, 255), 10)
                new_frame[x_p+x-x_offset][y_p+y-y_offset] = [0,0,255]
                #frame[x_p + x-20:x_p + x+20,y_p + y-20:y_p + y+20]=[0,0,0]
                y_new = y_p+y-y_offset
                x_new = x_p+x-x_offset
    """
    return y_new, x_new #, new_frame




In [2]:

def get_trans(x,y, h):
    x_p = (h[0,0] * x + h[0,1] *y + h[0,2] )/(h[2,0]* x + h[2,1]* y + h[2,2])
    y_p = (h[1,0]*x + h[1,1]*y + h[1,2])/(h[2,0]*x + h[2,1]* y + h[2,2])

    return x_p,y_p

In [3]:
def grey(img1):
    down = 40
    B,G,R = cv2.split(img1)
    grey =0.2989 *R + 0.5870 *G + 0.1140 *B
    img1[:,:,0] = grey-down# get blue channel
    img1[:,:,1] = grey-down # get green channel
    img1[:,:,2] = grey-down  # get red channel
    return img1

In [4]:
import cv2
cap = cv2.VideoCapture("KandinskyBook.mp4") # video


# capture one frame
ret,frame = cap.read()

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)+0.5) 
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)+0.5)

x_list = [207,379,382,205]
y_list = [542,540,765,768]

yumi = cv2.imread('yumi.jpg')
yumi = grey(yumi)
yumi_points = np.array([(0,0), (743,0),(743,887),(0, 887)])

fps = int(cap.get(cv2.CAP_PROP_FPS)) 
out = cv2.VideoWriter('yumi_F.avi',cv2.VideoWriter_fourcc(*'MJPG'), fps, (width,height))


[ERROR:0@2.387] global cap.cpp:597 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.7.0) /private/var/folders/qx/953trp_56zs_kg4jq_5138gm0000gn/T/pip-install-7gkqnbkl/opencv-python_29b16a6fc84f41dc8ae76a97b840b22d/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): yumi_F.avi in function 'icvExtractPattern'




In [ ]:

while True:
    # grab a frame
    ret,frame = cap.read() 
    
    if ret == True: 
        new_frame = frame.copy()
        new_x_list =[]
        new_y_list =[]
        for x_p, y_p in zip(x_list, y_list):
            y_p, x_p = find_corner(y_p, x_p, frame)
            new_x_list.append(x_p)
            new_y_list.append(y_p)
            #new_frame[y_p - 6 :y_p + 6, x_p-6: x_p+6 ]=[0,0,255]
        #cv2.imshow('cam shift tracking demo',new_frame)
        x_list = new_x_list
        y_list = new_y_list
        
        
        
        
        pts_dst = np.array(list(zip(x_list,y_list)))
        h, status = cv2.findHomography(yumi_points, pts_dst)
        for y in range(yumi.shape[0]):
            for x in range(yumi.shape[1]):
                x_p, y_p = get_trans(x,y, h)
                y_p = int(y_p)
                x_p = int(x_p)
                new_frame[y_p,x_p] = yumi[y,x]
        cv2.imshow('cam shift tracking demo',new_frame)
        
        out.write(new_frame)
        
        #cv2.imshow('cam shift tracking demo',frame)
        if cv2.waitKey(30) & 0xFF == 27: # wait 1 ms and exit is ESC is pressed
            break
        
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()